In [1]:
import numpy as np
import pandas as pd
%matplotlib qt
import sys
import os
import warnings
from tqdm import tqdm
warnings.filterwarnings("ignore", category=RuntimeWarning)
sys.path.append('../../')
import Utils.Preprocess as ut
project_path = os.path.abspath(os.path.relpath('../../../../', os.getcwd()))
chb_root_path=os.path.join(project_path,'Dataset/CHB-MIT')
constrain_path=os.path.join(project_path,'BilinearNetwork/Data/Constraint')
exclude_patient=pd.read_csv(os.path.join(constrain_path,'exclude_Patient.csv'))
seizure_table=pd.read_csv(os.path.join(constrain_path,'seizure_preictal_summary.csv'))

In [2]:
def extract_epochs(raw_path,start,stop):
    raw_T=ut.PreprocessTool(raw_path,start=start,stop=stop,duration=5)
    raw_T.do_preprocess().overlap_events().cut_epochs()
    epochs_data=raw_T.get_epochs().get_data()
    stft_data=raw_T.get_epochs_stft()
    del raw_T
    return epochs_data,stft_data

In [3]:
local_epochs_time=[]
local_epochs_stft=[]
local_epoch_time=None
prelix=None

for row in tqdm(seizure_table.iterrows(),total=seizure_table.shape[0]):
    prelix_old=prelix
    file_name=row[1]['File Name']
    prelix=file_name.split('_')[0]

    if prelix_old!=prelix and prelix!='chb01':
        # save_path_time='%s_preictal.npy'%os.path.join(os.path.join(project_path,'BilinearNetwork/Data/PreprocessedData/CHB-MIT/Time'),prelix_old)
        save_path_stft='%s_preictal'%os.path.join(os.path.join(project_path,'BilinearNetwork/Data/PreprocessedData/CHB-MIT/STFT'),prelix_old)
        
        if len(local_epochs_stft)>0:
            # local_epochs_time=np.concatenate(local_epochs_time,axis=0)
            local_epochs_stft=np.concatenate(local_epochs_stft,axis=0).astype(np.float32)
            
            print(local_epochs_stft.shape)
            print('-------')
            
            # np.save(save_path_time,local_epochs_time)
            label=np.ones(local_epochs_stft.shape[0]).astype(np.float32)
            np.savez(save_path_stft,data=local_epochs_stft,label=label)
            print('Save %s'%save_path_stft)
        else:
            continue

        local_epochs_time=[]
        local_epochs_stft=[]


    chb_patient_path=os.path.join(chb_root_path,prelix.rstrip('a').rstrip('b'))
    process_file_path=os.path.join(chb_patient_path,file_name)
    start=row[1]['Preictal Start Time']
    stop=row[1]['Preictal End Time']

    if prelix in list(exclude_patient['0']):

        continue
    if stop-start<5:
        continue
    try:
        _,local_epoch_stft=extract_epochs(process_file_path,start,stop)
    except ValueError:

        continue
    assert local_epoch_stft.shape[0]>0
    # local_epochs_time.append(local_epoch_time)
    
    local_epochs_stft.append(local_epoch_stft)
    
    